# 🔄 CarRacing AI 추가 학습 노트북

이미 학습된 best model을 불러와서 추가로 학습시킵니다.

**사용 시점**: 이전 학습이 완료되고, 더 나은 성능을 원할 때

**목표**:
- 기존 400점대 → 600점 이상
- 잔디밭 회복 능력 향상
- 코너링 안정성 증가

---

## 1. GPU 설정 확인

**중요**: 런타임 > 런타임 유형 변경 > 하드웨어 가속기를 **GPU**로 설정하세요!

In [ ]:
# GPU 확인
!nvidia-smi

## 2. Google Drive 마운트

이전에 학습한 모델을 불러오기 위해 Google Drive를 마운트합니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Google Drive 경로
DRIVE_BASE_PATH = '/content/drive/MyDrive/CarRacing_AI'

print("✅ Google Drive 마운트 완료!")
print(f"📁 모델 경로: {DRIVE_BASE_PATH}")

## 3. 패키지 설치

In [ ]:
# SWIG 먼저 설치
!apt-get update -qq
!apt-get install -y swig build-essential python3-dev

# Box2D 먼저 설치
!pip install box2d-py

# 나머지 패키지 설치
!pip install gymnasium stable-baselines3[extra] shimmy moviepy pygame -q

print("✅ 패키지 설치 완료!")

## 4. 라이브러리 임포트

In [ ]:
import os
import glob
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from gymnasium.wrappers import RecordVideo

print("✅ 라이브러리 임포트 완료!")

## 5. 기존 모델 확인

저장된 모델들을 확인합니다.

In [ ]:
MODEL_DIR = f"{DRIVE_BASE_PATH}/models"

print("="*60)
print("📂 저장된 모델 목록")
print("="*60)

# 모든 모델 파일 찾기
model_files = glob.glob(f"{MODEL_DIR}/*.zip")

if model_files:
    for i, model_file in enumerate(sorted(model_files), 1):
        file_name = os.path.basename(model_file)
        file_size = os.path.getsize(model_file) / (1024 * 1024)
        print(f"{i}. {file_name} ({file_size:.2f} MB)")
    
    print(f"\n✅ 총 {len(model_files)}개의 모델 발견")
    print("\n💡 권장: 'best_model.zip' 사용 (가장 높은 성능)")
else:
    print("❌ 저장된 모델이 없습니다!")
    print("먼저 colab_train.ipynb를 실행하여 모델을 학습시켜주세요.")

## 6. 추가 학습 설정

**성능 개선 전략**:
- 탐험 계수(ent_coef) 증가: 새로운 행동 시도
- 학습률(lr) 감소: 기존 지식 보존하며 미세 조정
- 추가 100,000 스텝 학습

In [ ]:
# 추가 학습 설정
ADDITIONAL_TIMESTEPS = 100000  # 추가로 학습할 스텝
CHECKPOINT_FREQ = 25000         # 체크포인트 저장 빈도
VIDEO_EVAL_FREQ = 25000         # 영상 녹화 빈도 (4회)
MODEL_NAME = "car_racing_driver_continued"

# 개선된 하이퍼파라미터
LEARNING_RATE = 0.00005  # 기존 0.0001에서 감소 (미세 조정)
ENT_COEF = 0.02          # 기존 0.01에서 증가 (더 많은 탐험)

# 디렉토리 설정
CHECKPOINT_DIR = f"{DRIVE_BASE_PATH}/checkpoints_continued"
LOG_DIR = f"{DRIVE_BASE_PATH}/logs_continued"
VIDEO_DIR = f"{DRIVE_BASE_PATH}/training_videos_continued"

# 디렉토리 생성
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

print("✅ 추가 학습 설정 완료!")
print(f"\n⚙️ 설정:")
print(f"  - 추가 학습 스텝: {ADDITIONAL_TIMESTEPS:,}")
print(f"  - 학습률: {LEARNING_RATE} (기존보다 낮춤)")
print(f"  - 탐험 계수: {ENT_COEF} (기존보다 높임)")
print(f"\n📁 저장 위치:")
print(f"  - 체크포인트: {CHECKPOINT_DIR}")
print(f"  - 로그: {LOG_DIR}")
print(f"  - 영상: {VIDEO_DIR}")
print(f"\n💡 전략: 기존 지식을 보존하며 새로운 행동 패턴 학습")

## 7. 환경 생성

In [ ]:
print("🏎️ 환경 초기화 중...")

def make_env():
    env = gym.make("CarRacing-v3", continuous=True)
    env = Monitor(env)
    return env

env = DummyVecEnv([make_env])

print("✅ 환경 초기화 완료!")

## 8. Best Model 로드

**중요**: 기존에 학습된 best_model을 불러옵니다.

In [ ]:
print("🤖 기존 모델 로딩 중...")

# Best model 경로
BEST_MODEL_PATH = f"{MODEL_DIR}/best_model"

try:
    # 기존 모델 로드
    model = PPO.load(
        BEST_MODEL_PATH,
        env=env,
        tensorboard_log=LOG_DIR
    )
    
    print(f"✅ 모델 로드 완료: {BEST_MODEL_PATH}.zip")
    
    # 하이퍼파라미터 업데이트 (추가 학습용)
    model.learning_rate = LEARNING_RATE
    model.ent_coef = ENT_COEF
    
    print(f"\n⚙️ 하이퍼파라미터 업데이트:")
    print(f"  - Learning Rate: {LEARNING_RATE}")
    print(f"  - Entropy Coefficient: {ENT_COEF}")
    print(f"\n💡 기존 지식을 유지하며 새로운 패턴을 학습합니다!")
    
except FileNotFoundError:
    print(f"❌ 모델을 찾을 수 없습니다: {BEST_MODEL_PATH}.zip")
    print(f"\n다음을 확인하세요:")
    print(f"1. 경로가 올바른지 확인")
    print(f"2. colab_train.ipynb를 먼저 실행했는지 확인")
    print(f"3. 위의 '5. 기존 모델 확인' 셀에서 모델 파일명 확인")
    raise

## 9. 콜백 설정

In [ ]:
# 체크포인트 콜백
checkpoint_callback = CheckpointCallback(
    save_freq=CHECKPOINT_FREQ,
    save_path=CHECKPOINT_DIR,
    name_prefix=MODEL_NAME
)

# 평가 환경 생성
def make_eval_env():
    eval_env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
    eval_env = Monitor(eval_env)
    eval_env = RecordVideo(
        eval_env,
        video_folder=VIDEO_DIR,
        episode_trigger=lambda x: True,
        name_prefix="continued-training"
    )
    return eval_env

eval_env = DummyVecEnv([make_eval_env])

# 평가 콜백
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=MODEL_DIR,
    log_path=LOG_DIR,
    eval_freq=VIDEO_EVAL_FREQ,
    n_eval_episodes=2,
    deterministic=True,
    render=False,
    verbose=1
)

print("✅ 콜백 설정 완료!")
print(f"\n📊 평가 설정:")
print(f"  - 평가 빈도: 매 {VIDEO_EVAL_FREQ:,} 스텝")
print(f"  - 평가 에피소드: 2개")
print(f"  - 예상 영상 개수: 약 8개 (4회 평가 × 2 에피소드)")

## 10. 추가 학습 시작

**학습 목표**:
- 기존 400점대 → 600점 이상
- 잔디밭에서 트랙으로 복귀하는 능력 향상
- 코너링 실수 후 회복 능력 향상

**예상 소요 시간**: 약 30~40분 (100,000 스텝)

In [ ]:
print("="*60)
print("🔄 추가 학습 시작!")
print("="*60)
print(f"추가 학습 스텝: {ADDITIONAL_TIMESTEPS:,}")
print(f"체크포인트 저장: 매 {CHECKPOINT_FREQ:,} 스텝")
print(f"영상 녹화: 매 {VIDEO_EVAL_FREQ:,} 스텝")
print(f"\n전략: 기존 지식 유지 + 새로운 행동 탐험")
print(f"목표: 400점 → 600점 이상")
print("="*60 + "\n")

try:
    model.learn(
        total_timesteps=ADDITIONAL_TIMESTEPS,
        callback=[checkpoint_callback, eval_callback],
        reset_num_timesteps=False  # ← 중요: 기존 타임스텝 이어서 계속
    )
    
    print("\n" + "="*60)
    print("🎉 추가 학습 완료!")
    print("="*60)
    
except KeyboardInterrupt:
    print("\n\n⚠️ 학습이 중단되었습니다.")

## 11. 최종 모델 저장

In [ ]:
# 추가 학습 완료 모델 저장
final_model_path = os.path.join(MODEL_DIR, f"final_{MODEL_NAME}")
model.save(final_model_path)
print(f"💾 최종 모델 저장: {final_model_path}.zip")

# 환경 종료
env.close()
eval_env.close()

# 저장된 파일 확인
checkpoints = glob.glob(f"{CHECKPOINT_DIR}/*.zip")
videos = glob.glob(f"{VIDEO_DIR}/*.mp4")

print(f"\n✅ 저장된 체크포인트: {len(checkpoints)}개")
print(f"✅ 저장된 영상: {len(videos)}개")
print(f"✅ 최종 모델: {final_model_path}.zip")
print(f"\n📁 Google Drive 위치: MyDrive/CarRacing_AI/")
print("\n🎉 추가 학습 완료! 이제 colab_test.ipynb로 테스트하세요!")

## 12. 학습 영상 확인

In [ ]:
from IPython.display import Video, display

video_files = sorted(glob.glob(f"{VIDEO_DIR}/*.mp4"))

print("="*60)
print("🎥 추가 학습 중 녹화된 영상")
print("="*60)

if video_files:
    for i, video_file in enumerate(video_files, 1):
        file_size = os.path.getsize(video_file) / (1024 * 1024)
        print(f"{i}. {os.path.basename(video_file)} ({file_size:.2f} MB)")
    
    print(f"\n✅ 총 {len(video_files)}개의 영상")
    print("\n💡 이전 영상과 비교하여 개선 정도를 확인하세요!")
else:
    print("❌ 녹화된 영상이 없습니다.")

## 13. 첫 번째 영상 재생 (추가 학습 초기)

In [ ]:
if video_files:
    print(f"🎬 첫 번째 영상: {os.path.basename(video_files[0])}")
    print("💡 추가 학습 초기 단계\n")
    display(Video(video_files[0], width=600))
else:
    print("❌ 영상이 없습니다.")

## 14. 마지막 영상 재생 (추가 학습 완료)

In [ ]:
if video_files and len(video_files) > 1:
    print(f"🎬 마지막 영상: {os.path.basename(video_files[-1])}")
    print("💡 추가 학습 완료 단계\n")
    display(Video(video_files[-1], width=600))
elif video_files:
    print("영상이 1개뿐입니다.")
else:
    print("❌ 영상이 없습니다.")

---

## 다음 단계

### 1. 성능 테스트
`colab_test.ipynb`를 실행하여 개선된 성능을 확인하세요:
- best_model은 자동으로 업데이트됨
- 기대 점수: 600점 이상

### 2. 비교 분석
- 이전 평균: ~714점 (461/780/901)
- 개선 목표: 최저 점수 상향 (461 → 600+)
- 일관성 향상: 편차 감소

### 3. 추가 개선이 필요하다면
- 이 노트북을 다시 실행 (추가 100k 스텝)
- ENT_COEF를 더 높이기 (0.02 → 0.03)
- 더 많은 스텝 학습 (200k 추가)

### 💡 참고
- `reset_num_timesteps=False`: 기존 학습 기록 이어가기
- 학습률 감소: 기존 지식 보존
- 탐험 계수 증가: 새로운 행동 패턴 발견

---

**만든이**: Claude Code